In [ ]:
import math
from collections import defaultdict
from datetime import datetime
from sqlalchemy import sql
from database_utils import Session; session = Session();
from models.models import Appointment, Education, Judge, Court, YearParty, Congress
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:

def get_judge_count_query(court_type_select=None, court_name_select=None):

    join_conditions = [
        # Join condition for if the judge was serving that year
        sql.and_(
            Appointment.start_year < YearParty.year + 2,
            sql.or_(
                Appointment.end_year >= YearParty.year , Appointment.end_year.is_(None)
            )
        ),
        # Join condition for party
        YearParty.party == Appointment.party_of_appointing_president,
    ]

    if court_type_select:
        join_conditions.append(Appointment.court_type.in_([court_type_select]))

    if court_name_select:
        join_conditions.append(Appointment.court_name.in_([court_name_select]))

    return (
        session
        .query(
            YearParty.year,
            YearParty.party,
            sql.func.count(Appointment.start_year).label('count'),
        )
        .outerjoin(
            Appointment,
            sql.and_(*join_conditions)
        )
        .group_by(YearParty.year, YearParty.party)
        .order_by(YearParty.year)
    )

def get_start_count_query(court_type_select=None, court_name_select=None):
    join_conditions = [
        # Need to have started / left in that congress or after
        Appointment.start_year >= YearParty.year,
        # Need to have started /  before the start of the next congress
        Appointment.start_year < YearParty.year + 2,
        # Join condition for party
        YearParty.party == Appointment.party_of_appointing_president,
    ]

    if court_type_select:
        join_conditions.append(Appointment.court_type.in_([court_type_select]))

    if court_name_select:
        join_conditions.append(Appointment.court_name.in_([court_name_select]))

    return (
        session
        .query(
            YearParty.year,
            YearParty.party,
            sql.func.count(Appointment.start_year).label('count'),
        )
        .outerjoin(
            Appointment,
            sql.and_(*join_conditions)
        )
        .group_by(YearParty.year, YearParty.party)
        .order_by(YearParty.year)
    )

def get_end_count_query(court_type_select=None, court_name_select=None):
    join_conditions = [
        # Need to have started / left in that congress or after
        Appointment.end_year >= YearParty.year - 2,
        # Need to have started /  before the start of the next congress
        Appointment.end_year < YearParty.year,
        # Join condition for party
        YearParty.party == Appointment.party_of_appointing_president,
    ]

    if court_type_select:
        join_conditions.append(Appointment.court_type.in_([court_type_select]))

    if court_name_select:
        join_conditions.append(Appointment.court_name.in_([court_name_select]))

    return (
        session
        .query(
            YearParty.year,
            YearParty.party,
            sql.func.count(Appointment.start_year).label('count'),
        )
        .outerjoin(
            Appointment,
            sql.and_(*join_conditions)
        )
        .group_by(YearParty.year, YearParty.party)
        .order_by(YearParty.year)
    )

start_query = get_start_count_query().subquery('start_query')
end_query = get_end_count_query().subquery('end_query')
count_query = get_judge_count_query().subquery('count_query')

full_query = (
    session
    .query(
        count_query.c.year,
        count_query.c.party,
        count_query.c.count.label('n_judges'),
        start_query.c.count.label('n_appointed'),
        end_query.c.count.label('n_terminated'),
    )
    .join(
        start_query,
        sql.and_(
            start_query.c.year == count_query.c.year,
            start_query.c.party == count_query.c.party,
        )
    )
    .join(
        end_query,
        sql.and_(
            end_query.c.year == count_query.c.year,
            end_query.c.party == count_query.c.party,
        )
    )
    .order_by(count_query.c.year)
)

In [ ]:
party_counts_dict = defaultdict(lambda: defaultdict(list))
years = set() ## set because of dups
for row in full_query:
    years.add(row.year)
    party_counts_dict[row.party]['n_judges'].append(row.n_judges) 
    party_counts_dict[row.party]['n_appointed'].append(row.n_appointed) 
    party_counts_dict[row.party]['n_terminated'].append(row.n_terminated) 
years = sorted(years)

In [ ]:
fig = go.Figure()
for party, counts_dict in party_counts_dict.items():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=counts_dict['n_judges'],
            mode='markers',
            error_y=dict(
                type='data',
                symmetric=False,
                array=counts_dict['n_appointed'],
                arrayminus=counts_dict['n_terminated'])
            )
    )
fig.show()

In [ ]:
169 - 286

y1 = {
    1377161,
    1378371,
    1384356,
    1384461,
    1385496,
    1385696,
    1387171,
    1388881,
    1389571,
    1393071,
}

y2 = {
    1377161,
    1377841,
    1378371,
    1378746,
    1380136,
    1384356,
    1384461,
    1385496,
    1385656,
    1385696,
    1387081,
    1387171,
    1387406,
    1388701,
    1388881,
    1389571,
    1390086,
    1393016,
    1393071,
}

In [ ]:
y1.difference(y2)

In [ ]:
y2.difference(y1)

In [ ]:
286 - 169